# Отображение файла в память (Memory mapping)

* Работа с содержимым файла, как будто он весь загружен в память ( независимо от их реальных размеров)
* Загрузка данных из файла происходит по необходимости, при этом не используемые страницы памяти освобождаются
* Улучшается производительность, за счет оптимизаций IO операций на уровне ОС

## Модуль [mmap](https://docs.python.org/3/library/mmap.html)

* Работа с содержимым файла как с `bytearray`.

In [29]:
with open("hello.txt", "wb") as f:
    f.write(b"Hello Python!\n")

In [30]:
import mmap

with open("hello.txt", "r+b") as f:
    # memory-map the file, size 0 means whole file
    mm = mmap.mmap(f.fileno(), 0)
    # read content via standard file methods
    print(mm.readline())  # prints b"Hello Python!\n"
    # read content via slice notation
    print(mm[:5])  # prints b"Hello"
    # update content using slice notation;
    # note that new content must have same size
    mm[6:] = b" world!\n"
    # ... and read again using standard file methods
    mm.seek(0)
    print(mm.readline())  # prints b"Hello  world!\n"
    # close the map
    mm.close()

b'Hello Python!\n'
b'Hello'
b'Hello  world!\n'


In [6]:
with mmap.mmap(-1, 13) as mm:
    mm.write(b"Hello world!")
    print(mm[:])

b'Hello world!\x00'


## Numpy memory mapping

In [1]:
import numpy as np

In [31]:
(4*1024*1024*1024/8)**0.5

23170.47500592079

### Создаем большой массив

In [13]:
shape = (20_000, 20_000)
fp = np.memmap("big_array.dat", dtype="d", mode="w+", shape=shape)


In [14]:
data = np.ones(20000, "d")
for i in range(20000):
    data *= i
    fp[i, :] = data
    fp.flush()

In [15]:
del data

In [23]:
!pip install psutil

In [2]:
import psutil
import gc
import os

In [3]:
process = psutil.Process(os.getpid())
print(process.memory_info().rss/1024/1024)

70.21875


In [28]:
gc.collect()

737

In [5]:
shape = (20_000, 20_000)
fp = np.memmap("big_array.dat", dtype="d", mode="r", shape=shape)

In [6]:
print(process.memory_info().rss/1024/1024)

70.65234375


In [7]:
data = fp[15000:, 15000]
data.sum()

0.0

In [8]:
print(process.memory_info().rss/1024/1024)

382.38671875


In [10]:
del fp
del data

In [12]:
print(process.memory_info().rss/1024/1024)

411.47265625


In [13]:
gc.collect()


1386

In [14]:
print(process.memory_info().rss/1024/1024)


99.1796875
